This experiment is done to show that plain Python is faster than Pandas if not comparable. It is done in python 3.12 and pandas 2.2.2:
```bash
conda create -n fairness-panda python=3.12 pandas=2.2.2 jupyter
conda activate fairness-panda
pip install fairness-checker
```

In [1]:
import pandas as pd
from fairness_checker import *

In [2]:
c = fairness_csv_checker("compas-scores-two-years.csv", verbose=False)
race = 'Native American'
sex = 'Male'
degree = 'M'
age = 'Greater than 45'

Using plain Python, this takes about 0.3 seconds:

In [3]:
for _ in range(100):
    c.disparate_impact(0.8, lambda row: row['sex'] == sex, lambda row: row['score_text'] in {'Medium', 'High'})

Using the same row predicate with Pandas, this takes around 12 seconds:

In [4]:
df = pd.read_csv("compas-scores-two-years.csv")
privileged_predicate = lambda row: row['sex'] == sex
positive_predicate = lambda row: row['score_text'] in {'Medium', 'High'}
for _ in range(100):
    privileged = df[df.apply(lambda row: privileged_predicate(row), axis=1)]
    unprivileged = df[df.apply(lambda row: not privileged_predicate(row), axis=1)]

    privileged_Y_result = privileged[privileged.apply(lambda row: positive_predicate(row), axis=1)]
    unprivileged_Y_result = unprivileged[unprivileged.apply(lambda row: positive_predicate(row), axis=1)]

    privileged_percentage = len(privileged_Y_result) / len(privileged)
    unprivileged_percentage = len(unprivileged_Y_result) / len(unprivileged)

    measure = unprivileged_percentage / privileged_percentage

Using the column predicate with Pandas, this takes around 1 seconds:

In [5]:
df = pd.read_csv("compas-scores-two-years.csv")
privileged_predicate = lambda x: x == sex
positive_predicate = lambda x: x in {'Medium', 'High'}
for _ in range(100):
    privileged = df[df['sex'].apply(lambda row: privileged_predicate(row))]
    unprivileged = df[df['sex'].apply(lambda row: not privileged_predicate(row))]

    privileged_Y_result = privileged[privileged['score_text'].apply(lambda row: positive_predicate(row))]
    unprivileged_Y_result = unprivileged[unprivileged['score_text'].apply(lambda row: positive_predicate(row))]

    privileged_percentage = len(privileged_Y_result) / len(privileged)
    unprivileged_percentage = len(unprivileged_Y_result) / len(unprivileged)

    measure = unprivileged_percentage / privileged_percentage